In [5]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd
from tabulate import tabulate

# Trainer
from src.trainer.trainer import Trainer

# Models
from src.models.mlp import MLP
from src.models.p4_allcnn import P4AllCNNC
from src.models.fA_p4_allcnn import fA_P4AllCNNC
from src.models.p4m_allcnn import  P4MAllCNNC
from src.models.fA_p4m_allcnn import fA_P4MAllCNNC
from src.models.p4m_resnet import  P4MResNet
from src.models.fA_p4_resnet import fA_P4MResNet


## CIFAR10 dataset

In [6]:
root_path = "../data"
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])
transform_test = transforms.ToTensor()

full_train = datasets.CIFAR10(root=root_path, train=True, download=True, transform=transform_train)

train_size = int(0.9 * len(full_train))
val_size = len(full_train) - train_size
train_set, val_set = random_split(full_train, [train_size, val_size])
test_set = datasets.CIFAR10(root=root_path, train=False, download=True, transform=transform_test)


batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [4]:
img, label = full_train[0]
print(img.shape)

torch.Size([3, 32, 32])


## Table generation

In [10]:
# Init training variables
num_epochs = 3 #10
num_iterations = 1#3
log_dir = "../logs"
accuracies = {
    "mlp": [],
    "p4_allcnn": [],
    "fA_p4_allcnn": [],
    "p4m_allcnn": [],
    "fA_p4m_allcnn": [],
    "p4m_resnet": [],
    "fA_p4_resnet": []
}

# ----- Init model paramerters -----

## MLP parameters:
input_size = 3 * 32 * 32
output_size = 10
hidden_sizes = [128, 64]

## ---------------------------------

for it in range(num_iterations):

    # ----- Models initialization -----
    mlp_model = MLP(
        input_size = input_size,
        hidden_sizes = hidden_sizes,
        output_size = output_size
    )

    p4_allcnn_model = P4AllCNNC()
    fA_p4_allcnn_model= fA_P4AllCNNC()
    p4m_allcnn_model = P4MAllCNNC()
    fA_p4m_allcnn_model = fA_P4MAllCNNC()

    p4m_resnet_model = P4MResNet()
    fA_p4_resnet_model= fA_P4MResNet()


    # ---------------------------------

    # ----- Other parameter initialization -----
    mlp_criterion = nn.CrossEntropyLoss()
    mlp_optimizer = optim.SGD(mlp_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    mlp_scheduler = optim.lr_scheduler.MultiStepLR(mlp_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # P4AllCNNC
    p4_allcnn_criterion = nn.CrossEntropyLoss()
    p4_allcnn_optimizer = optim.SGD(p4_allcnn_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    p4_allcnn_scheduler = optim.lr_scheduler.MultiStepLR(p4_allcnn_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # fA_P4AllCNNC
    fA_p4_allcnn_criterion = nn.CrossEntropyLoss()
    fA_p4_allcnn_optimizer = optim.SGD(fA_p4_allcnn_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    fA_p4_allcnn_scheduler = optim.lr_scheduler.MultiStepLR(fA_p4_allcnn_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # P4MAllCNNC
    p4m_allcnn_criterion = nn.CrossEntropyLoss()
    p4m_allcnn_optimizer = optim.SGD(p4m_allcnn_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    p4m_allcnn_scheduler = optim.lr_scheduler.MultiStepLR(p4m_allcnn_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # fA_P4MAllCNNC
    fA_p4m_allcnn_criterion = nn.CrossEntropyLoss()
    fA_p4m_allcnn_optimizer = optim.SGD(fA_p4m_allcnn_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    fA_p4m_allcnn_scheduler = optim.lr_scheduler.MultiStepLR(fA_p4m_allcnn_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # P4MResNet
    p4m_resnet_criterion = nn.CrossEntropyLoss()
    p4m_resnet_optimizer = optim.SGD(p4m_resnet_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    p4m_resnet_scheduler = optim.lr_scheduler.MultiStepLR(p4m_resnet_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # fA_P4MResNet
    fA_p4_resnet_criterion = nn.CrossEntropyLoss()
    fA_p4_resnet_optimizer = optim.SGD(fA_p4_resnet_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    fA_p4_resnet_scheduler = optim.lr_scheduler.MultiStepLR(fA_p4_resnet_optimizer, milestones=[200, 250, 300], gamma=0.1)


    # ------------------------------------------

    # Generate lists
    models = {
    "mlp": mlp_model,
    "p4_allcnn": p4_allcnn_model,
    "fA_p4_allcnn": fA_p4_allcnn_model,
    "p4m_allcnn": p4m_allcnn_model,
    "fA_p4m_allcnn": fA_p4m_allcnn_model,
    "p4m_resnet": p4m_resnet_model,
    "fA_p4_resnet": fA_p4_resnet_model
    }

    criterions = [
        mlp_criterion,
        p4_allcnn_criterion,
        fA_p4_allcnn_criterion,
        p4m_allcnn_criterion,
        fA_p4m_allcnn_criterion,
        p4m_resnet_criterion,
        fA_p4_resnet_criterion
    ]

    optimizers = [
        mlp_optimizer,
        p4_allcnn_optimizer,
        fA_p4_allcnn_optimizer,
        p4m_allcnn_optimizer,
        fA_p4m_allcnn_optimizer,
        p4m_resnet_optimizer,
        fA_p4_resnet_optimizer
    ]

    schedulers = [
        mlp_scheduler,
        p4_allcnn_scheduler,
        fA_p4_allcnn_scheduler,
        p4m_allcnn_scheduler,
        fA_p4m_allcnn_scheduler,
        p4m_resnet_scheduler,
        fA_p4_resnet_scheduler
    ]

    # -------------------------------------------

    trainer = Trainer(
        models = models,
        optimizers = optimizers,
        criterions = criterions,
        schedulers = schedulers,
        log_dir = log_dir
    )

    # Train models on dataset
    print("Training models...")
    trainer.train(
        num_epochs = num_epochs,
        train_loader = train_loader,
        val_loader = val_loader,
    )

    # Evaluate models on dataset
    print("Testing models...")
    test_accuracies = trainer.evaluate(test_loader = test_loader)

    for model_name, acc in test_accuracies.items():
        accuracies[model_name].append(acc)

# get final statistics
final_stats = {
    model_name: {
        "% Test error" : (1 - float(np.mean(values))) * 100,
        "% std" : float(np.std(values)) * 100,
        "Num Parameters" : sum(p.numel() for p in models[model_name].parameters()),
    }
    for model_name, values in accuracies.items()
}

Using device: cpu
Training models...
Training model : mlp


Training Epochs:   0%|          | 0/3 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
Training Epochs:   0%|          | 0/3 [00:05<?, ?it/s]  File "/Users/larabastos/miniconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/larabastos/miniconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/torchvision/__init__.py", line 10, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils  # usort:skip
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/torchvision/models/__init__.py", line 2, in <module>
    fro

KeyboardInterrupt: 

## Table

In [8]:
df = pd.DataFrame.from_dict(final_stats, orient='index')
df = df.round(2)
print("📊 Model Accuracy Summary in CIFAR10\n")
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

📊 Model Accuracy Summary in CIFAR10

╒═════╤════════════════╤═════════╤══════════════════╕
│     │   % Test error │   % std │   Num Parameters │
╞═════╪════════════════╪═════════╪══════════════════╡
│ mlp │          74.52 │    2.38 │           402250 │
╘═════╧════════════════╧═════════╧══════════════════╛
